# Progama brasileiro de imunização - COVID 19

## Objetivos

Transformar os dados brutos dos registro de vacinação para um modelo analítico, disponibilizando para consulta pública.

Utilizaremos os seguintes conjuntos de dados:
* [Registros de Vacinação COVID19 do Ministério da Saúde](https://dados.gov.br/dataset/covid-19-vacinacao/resource/ef3bd0b8-b605-474b-9ae5-c97390c197a8?inner_span=True)
    * O processamento será feito por Unidade Federativa, para fins de otimização

**Os dados serão extraídos dos arquivos CSV fornecidos pelo Ministério da Saúde, e carregados em um conjunto de tabelas analíticas, seguindo o modelo [Star Schema](https://en.wikipedia.org/wiki/Star_schema).**.

## Enviroment setup

In [ ]:
# Import necessary libraries
import pandas as pd
import uuid
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf
import configparser

In [ ]:
# Read config file
config = configparser.ConfigParser()
config.read_file(open('dl.cfg'))

INPUT_DATA = config['LOCAL']['INPUT_DATA']
INPUT_DATA_VACCINES = config['LOCAL']['INPUT_DATA_VACCINES']
OUTPUT_DATA = config['LOCAL']['OUTPUT_DATA']
DATA_COLUMNS = config['COMMON']['DATA_COLUMNS']

In [ ]:
@udf(StringType())
def null_id_to_uuid (id):
    if id == None or id == "null":
        return str(uuid.uuid4().hex)
    return id

In [ ]:
def write_parquet(df, parquet_name):
    parquet_path = OUTPUT_DATA + f'{parquet_name}.parquet'
    df.write.mode("overwrite").parquet(parquet_path)
    print(f'Writing {parquet_name} Table DONE.')


In [ ]:
def read_parquet(parquet_name):
    parquet_path = OUTPUT_DATA + f'{parquet_name}.parquet'
    return spark.read.parquet(parquet_path)

In [ ]:
def check_nulls(df, columns_list, expected_value):
    df.createOrReplaceTempView("viewcheck")
    sql_check = f"SELECT COUNT(*) FROM viewcheck WHERE 1 <> 1 {''.join([' OR ' + c + ' IS NULL ' for c in columns_list])}"
    
    dfcheck = spark.sql(sql_check)
    
    value_check = dfcheck.collect()[0][0]
    
    return value_check == expected_value

In [ ]:
def check_has_content(df):
    return df.count() > 0

In [ ]:
# Spark session
spark = SparkSession \
        .builder\
        .appName("Brasil - Programa de imunização - COVID-19")\
        .getOrCreate()

In [ ]:
dados_imunizacao = spark.read.csv(INPUT_DATA_VACCINES, sep=';', header=True)

dados_imunizacao.printSchema()

## Step 2: Explore and Assess the Data
In this step we need:
* Explore the data to identify data quality issues, like missing values, duplicate data, etc.
* Document steps necessary to clean the data

In [ ]:
# Read the data dictionary from JSON and extract the valid columns
col_names = pd.read_json(DATA_COLUMNS, typ='series')
colunas_utilizadas = col_names.index
colunas_utilizadas

In [ ]:
# Get the difference between the dataframe colums and the valid columns
colunas_naoutilizadas = list(set(dados_imunizacao.columns) - set(colunas_utilizadas))

colunas_naoutilizadas

In [ ]:
# Remove unused columns from dataframe
dados_imunizacao = dados_imunizacao.drop(*colunas_naoutilizadas)
dados_imunizacao.printSchema()

In [ ]:
# Replace the null values
dados_imunizacao = dados_imunizacao.fillna(\
    {\
        'vacina_categoria_codigo': 0, \
        'vacina_categoria_nome': 'N/A', \
        'vacina_grupoatendimento_nome': 'N/A', \
        'paciente_enumsexobiologico': 'N/A',\
        'paciente_endereco_nmmunicipio': 'N/A', \
        'paciente_endereco_nmpais': 'N/A', \
        'paciente_endereco_uf': 'N/A', \
        'estalecimento_nofantasia': 'N/A'
    })

In [ ]:
dados_imunizacao.printSchema()

In [ ]:
# Write staging table and read back
write_parquet(dados_imunizacao, 'staging_imunizacao')

dados_imunizacao = read_parquet('staging_imunizacao')

## Step 3: Define the Data Model
_Map out the conceptual data model and explain why you chose that model_

The data model is a star schema consisting of 5 Dimensions table and 1 Fact table:
  * Dimensions tables:
      * vaccines table: Vaccines and suppliers
      * health_institution table: Hospitals, Nursing home, Clinics 
      * category table: Priority groups
      * population_group table: Demograph group (professions, age group, ethnicity)
      * patient table: Demograph data (age, city, gender)
  * Fact table:
      * immunization table: Dimensions, First | second dose, date

![ER Data Model - Star Scheme](./docs/er-model-star.jpg)

_List the steps necessary to pipeline the data into the chosen data model_
* ETL starts the enviroment setup: imports, read config file, def functions and create Spark Session
* ETL script takes source data (Brazilian Government' dataset COVID-19 population immunization program)
* Raw data is read into dataframe and cleaned (remove unused columns, fill nulls) 
* For each dimension and fact table 
	* Create a temporary view table
	* Read data to new dataframe
    * Check data quality: key columns don't have nulls, each table has content
	* Create id/indexes (if necessary)
	* Write parquet files

## Step 4: Run ETL to Model the Data
* Create the data pipelines and the data model
* Include a data dictionary
* Run data quality checks to ensure the pipeline ran as expected
	* Integrity constraints on the relational database (e.g., unique key, data type, etc.)
	* Unit tests for the scripts to ensure they are doing the right thing
	* Source/count checks to ensure completeness

In [ ]:
# Create vaccines table
dados_imunizacao.createOrReplaceTempView("vacinas_df")
vacinas_df = spark.sql("""
    SELECT  DISTINCT vacina_codigo AS id, 
                     vacina_nome AS name, 
                     vacina_fabricante_nome AS supplier
    FROM vacinas_df
    ORDER BY supplier
""")

vacinas_df.printSchema()
vacinas_df.show()

In [ ]:
# Check data quality
if not check_nulls(vacinas_df, ['id', 'name', 'supplier'], 0): raise Exception('Null: Vaccines tables')
if not check_has_content(vacinas_df): raise Exception('No content: Vaccines table')

In [ ]:
# Write parquet file
write_parquet(vacinas_df, 'vacinas')

In [ ]:
# Create Health Institution table
dados_imunizacao.createOrReplaceTempView("estabelecimento_df")
estabelecimento_df = spark.sql("""
    SELECT DISTINCT estabelecimento_valor AS id,
                    estalecimento_nofantasia AS name,
                    estabelecimento_razaosocial AS organization,
                    estabelecimento_uf AS state,
                    estabelecimento_municipio_nome AS city
    FROM estabelecimento_df
    ORDER BY name
""")

estabelecimento_df.printSchema()
estabelecimento_df.show()

In [ ]:
# Check data quality
if not check_nulls(estabelecimento_df, ['id', 'name', 'organization', 'state', 'city'], 0): raise Exception('Null: Health institution table')
if not check_has_content(estabelecimento_df): raise Exception('No content: Health institution table')

In [ ]:
# Write parquet file
write_parquet(estabelecimento_df, 'estabelecimentos')

In [ ]:
# Create Category table
dados_imunizacao.createOrReplaceTempView("categorias_df")
categorias_df = spark.sql("""
    SELECT DISTINCT vacina_categoria_codigo AS id,
                    vacina_categoria_nome AS name
            FROM categorias_df
            ORDER BY name
""")

categorias_df.printSchema()
categorias_df.show()

In [ ]:
# Check data quality
if not check_nulls(categorias_df, ['id', 'name'], 0): raise Exception('Null: Category table')
if not check_has_content(categorias_df): raise Exception('No content: Category table')

In [ ]:
# Write parquet file
write_parquet(categorias_df, 'categorias')

In [ ]:
# Create Population Groups table
dados_imunizacao.createOrReplaceTempView("grupos_atendimento_df")
grupos_atendimento_df = spark.sql("""
    SELECT DISTINCT vacina_grupoatendimento_codigo AS id,
                    vacina_grupoatendimento_nome AS name
            FROM grupos_atendimento_df
        ORDER BY name
""")

grupos_atendimento_df.printSchema()
grupos_atendimento_df.show()

In [ ]:
# Check data quality
if not check_nulls(grupos_atendimento_df, ['id', 'name'], 0): raise Exception('Null: Population group table')
if not check_has_content(grupos_atendimento_df): raise Exception('No content: Population group table')

In [ ]:
# Write parquet file
write_parquet(grupos_atendimento_df, 'grupos')

In [ ]:
# Create Patient table
dados_imunizacao.createOrReplaceTempView("pacientes_df")
pacientes_df = spark.sql("""
    SELECT DISTINCT paciente_id AS id,
                    paciente_idade AS age,
                    paciente_datanascimento AS birthdate,
                    paciente_enumsexobiologico AS gender,
                    paciente_endereco_nmpais AS country,
                    paciente_endereco_uf AS state,
                    paciente_endereco_nmmunicipio AS city
            FROM pacientes_df
            WHERE paciente_id IS NOT NULL
            ORDER BY id
""")

pacientes_df.printSchema()
pacientes_df.show()

In [ ]:
# Check data quality
if not check_nulls(pacientes_df, ['id', 'age', 'birthdate', 'gender', 'country', 'state', 'city'], 0): raise Exception('Null: Patient table')
if not check_has_content(pacientes_df): raise Exception('No content: Patient table')

In [ ]:
# Write parquet file and get back to Spark:
write_parquet(pacientes_df, 'pacientes')

In [ ]:
# Get back Health Institution table from parquet
estabelecimento_df = read_parquet('estabelecimentos')

In [ ]:
# Create Patient table and write parquet files
dados_imunizacao.createOrReplaceTempView("dados_imunizacao")
vacinacao_df = spark.sql("""
    SELECT DISTINCT paciente_id AS patient_id,
            estabelecimento_valor AS health_institution_id,
            vacina_categoria_codigo AS category_id,
            vacina_grupoatendimento_codigo AS population_group_id,
            vacina_codigo AS vaccines_id,
            vacina_descricao_dose AS vaccines_dose,
            vacina_dataaplicacao AS jab_date
        FROM dados_imunizacao 
        WHERE paciente_id IS NOT NULL
        ORDER BY jab_date
""")

vacinacao_df.printSchema()

In [ ]:
# Check data quality
if not check_nulls(vacinacao_df, \
        ['patient_id', \
        'health_institution_id', \
        'category_id', \
        'population_group_id', \
        'vaccines_id', \
        'vaccines_dose', \
        'jab_date'], 0): raise Exception('Null: Immunization table')
if not check_has_content(vacinacao_df): raise Exception('No content: Immunization table')

In [ ]:
# Write parquet file and get back to Spark:
write_parquet(vacinacao_df, 'vacinacao')
vacinacao_df = read_parquet('vacinacao')

In [ ]:
vacinacao_df.show()

### Data dictionary

[Dict](./data-dictionary.json)
```
{
  "vaccines": {
    "description": "Vaccines and suppliers",
    "fields": {
      "id": "Identity field",
      "name": "Vaccine name",
      "supplier": "Laboratory/supplier"
    }
  },
  "health_institution": {
    "description": "Hospitals, Nursing home, Clinics",
    "fields": {
      "id": "Identity field",
      "name": "Institution name",
      "organization": "",
      "state": "Brazilian State",
      "city": "City name"
    }
  },
  "category": {
    "description": "Priority groups",
    "fields": {
      "id": "Identity field",
      "name": "Group name"
    }
  },
  "population_group": {
    "description": "Demograph group (professions, age group, ethnicity)",
    "fields": {
      "id": "Identity field",
      "name": "Group name"
    }
  },
  "patient": {
    "description": "Demograph data (age, city, gender)",
    "fields": {
      "id": "Identity field",
      "age": "Patient age (years)",
      "birthdate": "Birth date",
      "gender": "Male/Female",
      "country": "Country (Brazil)",
      "state": "Brazilian State",
      "city": "City name"
    }
  },
  "imunization": {
    "description": "Dimensions, First | second dose, date",
    "fields": {
      "patient_id": "(FK) Patient Identity",
      "health_institution_id": "(FK) Health institution Identity",
      "category_id": "(FK) Category group Identity",
      "population_group_id": "(FK) Population group Identity",
      "vaccines_id": "(FK) Vaccine Identity",
      "vaccines_dose": "First/Second dose",
      "jab_date": "Vaccine date"
    }
  }
}
```

## Step 5: Complete Project Write Up
Tools:
* Python
* Pandas
* Spark

These tools/technologies are apropriated to manipulate large dataset, processing in paralellized clusters

**ETL script should be run weekly basis**, or whenever the .gov.br update the datasets

How the script would approach the problem differently under the following scenarios:
* If the data was increased by 100x: _Use Spark Clustered to parallel the data load_
* If the pipelines were run on a daily basis by 7am: _Refactor the script to process only new informations. The [.gov.br API](https://dados.gov.br/dataset/covid-19-vacinacao/resource/97a8fbcf-941f-4d2e-91ba-dd467d5bdeac?inner_span=True) could be used to request the delta information_
* If the database needed to be accessed by 100+ people: _Store the parquet files on a cloud data lake_

### Sample queries

In [ ]:
vacinas_df = read_parquet('vaccines')

# Get Health institution by vaccines aplication
vacinas_df.createOrReplaceTempView("vdf")
estabelecimento_df.createOrReplaceTempView("hidf")
vacinacao_df.createOrReplaceTempView("idf")

health_institution_rank = spark.sql("""
    SELECT 
        CASE
            WHEN (GROUPING(hidf.name) = 1) THEN '# Total Institution'
            ELSE hidf.name
        END AS Institution,
        CASE
            WHEN (GROUPING(vdf.name) = 1) THEN '# Total Vaccines'
            ELSE vdf.name
        END AS Vaccine,
        COUNT(*) AS TOTAL
    FROM idf 
    INNER JOIN hidf ON idf.health_institution_id = hidf.id
    INNER JOIN vdf ON idf.vaccines_id = vdf.id
    GROUP BY CUBE(hidf.name, vdf.name)
    ORDER BY Institution, TOTAL DESC, Vaccine
""").toPandas()

health_institution_rank.style

In [ ]:
# Get vaccines aplication
vaccines_rank = spark.sql("""
    SELECT 
        CASE
            WHEN (GROUPING(vdf.supplier) = 1) THEN '# Total Supplier'
            ELSE vdf.supplier
        END AS Supplier,
        CASE
            WHEN (GROUPING(vdf.name) = 1) THEN '# Total Vaccines'
            ELSE vdf.name
        END AS Vaccine,
        COUNT(*) AS TOTAL
    FROM idf 
    INNER JOIN vdf ON idf.vaccines_id = vdf.id
    GROUP BY CUBE(vdf.supplier, vdf.name)
    ORDER BY Supplier, TOTAL DESC
""").toPandas()

vaccines_rank.style